<a href="https://colab.research.google.com/github/Arkenstone999/data-engineering-practice-for-me/blob/main/Getting_started_with_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

work through https://medium.com/@chsafouane/getting-started-with-pytorch-on-google-colab-811c59a656b6

The good news about Colab is that Google offers a Tesla K80 GPU. One can keep working with the GPU instance continuously up to 12 hours

We can also work through the tutorials on https://notebooks.azure.com/pytorch

In [ ]:
!pip3 install torch torchvision

    100% |████████████████████████████████| 519.5MB 34kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x592c6000 @  0x7fb7402bf1c4 0x46d6a4 0x5fcbcc 0x4c494d 0x54f3c4 0x553aaf 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54e4c8
    100% |████████████████████████████████| 61kB 20.4MB/s 
    100% |████████████████████████████████| 2.0MB 4.2MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


In [ ]:
import torch

In [ ]:
torch.__version__

'0.4.1'

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import math
import copy

In [14]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        # Ensure that the model dimension (d_model) is divisible by the number of heads
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"

        # Initialize dimensions
        self.d_model = d_model # Model's dimension
        self.num_heads = num_heads # Number of attention heads
        self.d_k = d_model // num_heads # Dimension of each head's key, query, and value

        # Linear layers for transforming inputs
        self.W_q = nn.Linear(d_model, d_model) # Query transformation
        self.W_k = nn.Linear(d_model, d_model) # Key transformation
        self.W_v = nn.Linear(d_model, d_model) # Value transformation
        self.W_o = nn.Linear(d_model, d_model) # Output transformation
def scaled_dot_product_attention(self, Q, K, V, mask=None):
        # Calculate attention scores
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)

        # Apply mask if provided (useful for preventing attention to certain parts like padding)
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)

        # Softmax is applied to obtain attention probabilities
        attn_probs = torch.softmax(attn_scores, dim=-1)

        # Multiply by values to obtain the final output
        output = torch.matmul(attn_probs, V)
        return output

def split_heads(self, x):
        # Reshape the input to have num_heads for multi-head attention
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)

In [15]:
def combine_heads(self, x):
        # Combine the multiple heads back to original shape
        batch_size, _, seq_length, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)

def forward(self, Q, K, V, mask=None):
        # Apply linear transformations and split heads
        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))

        # Perform scaled dot-product attention
        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)

        # Combine heads and apply output transformation
        output = self.W_o(self.combine_heads(attn_output))
        return output

In [16]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length):
        super(PositionalEncoding, self).__init__()

        pe = torch.zeros(max_seq_length, d_model)
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        self.register_buffer('pe', pe.unsqueeze(0))

    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

In [17]:
class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PositionWiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))

In [18]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask):
        attn_output = self.self_attn(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x

In [19]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.cross_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, enc_output, src_mask, tgt_mask):
        attn_output = self.self_attn(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout(attn_output))
        attn_output = self.cross_attn(x, enc_output, enc_output, src_mask)
        x = self.norm2(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm3(x + self.dropout(ff_output))
        return x

In [32]:
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout):
        super(Transformer, self).__init__()
        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_seq_length)

        self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.decoder_layers = nn.ModuleList([DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])

        self.fc = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)
def generate_mask(self, src, tgt):
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3)
        seq_length = tgt.size(1)
        nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool()
        tgt_mask = tgt_mask & nopeak_mask
        return src_mask, tgt_mask

def forward(self, src, tgt):
        src_mask, tgt_mask = self.generate_mask(src, tgt)
        src_embedded = self.dropout(self.positional_encoding(self.encoder_embedding(src)))
        tgt_embedded = self.dropout(self.positional_encoding(self.decoder_embedding(tgt)))

        enc_output = src_embedded
        for enc_layer in self.encoder_layers:
            enc_output = enc_layer(enc_output, src_mask)

        dec_output = tgt_embedded
        for dec_layer in self.decoder_layers:
            dec_output = dec_layer(dec_output, enc_output, src_mask, tgt_mask)

        output = self.fc(dec_output)

In [33]:
src_vocab_size = 1000  # Example: size of the source vocabulary
tgt_vocab_size = 1000  # Example: size of the target vocabulary
d_model = 512          # Example: dimension of the model
num_heads = 8          # Example: number of attention heads
num_layers = 6         # Example: number of encoder and decoder layers
d_ff = 2048          # Example: dimension of the feed-forward network
max_seq_length = 100   # Example: maximum sequence length
dropout = 0.1          # Example: dropout rate


In [34]:
transformer = Transformer(src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)

In [37]:
from transformers import (
    pipeline,
    AutoTokenizer,
    AutoModelForSequenceClassification,
    AutoModelForMaskedLM,
    AutoModelForQuestionAnswering,
    AutoModelForSeq2SeqLM,
    AutoModelForCausalLM,
)
import torch

In [38]:
def setup_pipelines():
    """Create high-level pipelines for common NLP tasks."""
    cps = {
        "sentiment": pipeline("sentiment-analysis"),
        "fill_mask": pipeline("fill-mask"),             # default: bert-base-uncased
        "qa": pipeline("question-answering"),            # default: distilbert-base-cased-distilled-squad
        "summarization": pipeline("summarization"),      # default: facebook/bart-large-cnn
        "translation": pipeline("translation_en_to_fr"), # default: t5-base
        "text_gen": pipeline("text-generation"),         # default: gpt2
    }
    return cps

In [39]:
pip install tf-keras

In [40]:
pip install transformers

In [41]:
cps = setup_pipelines()

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0


In [42]:
print("1) Sentiment Analysis")
print(cps["sentiment"]("I love this product! It's fantastic and works like a charm."))
print()

1) Sentiment Analysis
[{'label': 'POSITIVE', 'score': 0.9998868703842163}]



In [43]:
print("2) Fill-Mask")
print(cps["fill_mask"]("The capital of France is <mask>.", top_k=5))
print()

2) Fill-Mask
[{'score': 0.2703702449798584, 'token': 2201, 'token_str': ' Paris', 'sequence': 'The capital of France is Paris.'}, {'score': 0.055882930755615234, 'token': 12790, 'token_str': ' Lyon', 'sequence': 'The capital of France is Lyon.'}, {'score': 0.029897717759013176, 'token': 4612, 'token_str': ' Barcelona', 'sequence': 'The capital of France is Barcelona.'}, {'score': 0.02308131940662861, 'token': 12696, 'token_str': ' Monaco', 'sequence': 'The capital of France is Monaco.'}, {'score': 0.020979775115847588, 'token': 5459, 'token_str': ' Berlin', 'sequence': 'The capital of France is Berlin.'}]



In [44]:
context = (
    "The Eiffel Tower is an iron lattice tower in Paris, "
    "named after the engineer Gustave Eiffel."
)
print(cps["qa"]({
    "question": "Who is the Eiffel Tower named after?",
    "context": context
}))

{'score': 0.8997244834899902, 'start': 77, 'end': 91, 'answer': 'Gustave Eiffel'}


/usr/local/lib/python3.11/dist-packages/transformers/pipelines/question_answering.py:390: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(


In [45]:
print("4) Summarization")
article = (
    """You deserve an explanation, so please don't skip this 1-minute read. It's Monday, June 9, and our fundraiser won't last long.
    If you've lost count of how many times you've visited Wikipedia this year, we hope that means it's given you at least €2.75 of knowledge.
    Please join the 2% of readers who give what they can to help keep this valuable resource ad-free, up-to-date, and available for all.

    After nearly 25 years, Wikipedia is still the internet we were promised—an oasis of free and collaborative knowledge.
    By visiting Wikipedia today, you're choosing a free and fair internet: a space where you can find facts you need without
    being distracted by ads or the agendas of wealthy owners.

    Most readers don't donate, so your gift matters. If Wikipedia provides you with €2.75 of knowledge,
    please donate €2.75 right now—or consider a monthly gift to help all year long. Thank you."""
)

4) Summarization


In [46]:
print(cps["summarization"](article, max_length=50))

Your min_length=56 must be inferior than your max_length=50.
/usr/local/lib/python3.11/dist-packages/transformers/generation/utils.py:1633: UserWarning: Unfeasible length constraints: `min_length` (56) is larger than the maximum possible length (50). Generation will stop at the defined maximum length. You should decrease the minimum length and/or increase the maximum length.
  warnings.warn(


[{'summary_text': " It's Monday, June 9, and our fundraiser won't last long . Please join the 2% of readers who give what they can to help keep this valuable resource ad-free . After nearly 25 years, Wikipedia is still the internet"}]


In [47]:
print("5) Translation (EN→FR)")
print(cps["translation"]("How are you today?", max_length=40))

Both `max_new_tokens` (=256) and `max_length`(=40) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


5) Translation (EN→FR)
[{'translation_text': "Comment vous êtes-vous aujourd'hui?"}]


In [48]:
print("6) Text Generation")
print(cps["text_gen"]("Once upon a time, in a land far, far away", max_length=50, num_return_sequences=1))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


6) Text Generation
[{'generated_text': "Once upon a time, in a land far, far away, the people had been waiting for a king to take them, and he was waiting. This is what happened to the people of Egypt. They were waiting for a king to take them, and he was waiting.\n\nIn the beginning, when the people had already been waiting for a king to take them, they had never been able to meet any king who had done anything more to them than he had done: they had no king to meet.\n\nThen the sun came up, and we saw a great figure, like a great king, standing over a great mountain. It was the king of Babylon, who had come to the people of Egypt, and had given them the land. And the king of Babylon took his wife and son and wife and son, and brought them to the land of Egypt. And when they had come to where the king of Babylon was, they had not come to the land of Egypt. So the king of Babylon's power went out of the land of Egypt, and came out to the land of Egypt. And he gave them the land to the 